In [ ]:
!wget https://hf.co/danjacobellis/walloc/resolve/main/RGB_Li_12c_J3_nf8_v1.0.2.pth

In [ ]:
!wget https://hf.co/danjacobellis/walloc/resolve/main/RGB_Li_48c_J3_nf8_v1.0.2.pth

In [1]:
import io
import torch
import numpy as np
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from PIL import Image
from datasets import load_dataset
from datasets import Image as HFImage
from anls_star import anls_score
from fastprogress.fastprogress import progress_bar
from torchvision.transforms import ToPILImage, PILToTensor
from walloc import walloc
class Config: pass

In [2]:
def walloc_compress(sample):
    with torch.no_grad():
        img = sample['image'].resize((896,896),resample=Image.Resampling.LANCZOS).convert("RGB")
        x = PILToTensor()(img).to(torch.float)
        x = (x/255 - 0.5).unsqueeze(0).to(device)
        H, W = x.size(2), x.size(3)
        x_padded = walloc.pad(x,p=8)
        X = codec.wavelet_analysis(x_padded,codec.J)
        Y = codec.encoder(X)
        X_hat = codec.decoder(Y)
        x_hat = codec.wavelet_synthesis(X_hat,codec.J)
        x_hat = codec.clamp(x_hat)        
        x_hat = walloc.crop(x_hat, (H,W))
        rec = ToPILImage()(x_hat[0] + 0.5)
        buff = io.BytesIO()
        rec.save(buff, format='WEBP', lossless=True)
        rec_webp_bytes = buff.getbuffer()   
    return {
        'image': rec_webp_bytes,
        'question': sample['question'],
        'questionId': sample['questionId'],
    }

In [3]:
ds = load_dataset("danjacobellis/docvqa",split='test')

Resolving data files:   0%|          | 0/57 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/57 [00:00<?, ?it/s]

In [4]:
device = "cuda"
checkpoints = [
    'RGB_Li_12c_J3_nf8_v1.0.2.pth',
    'RGB_Li_48c_J3_nf8_v1.0.2.pth'
]
fic_ds = []
for cf in checkpoints:
    checkpoint = torch.load(cf,map_location="cpu",weights_only=False)
    codec_config = checkpoint['config']
    codec = walloc.Codec2D(
        channels = codec_config.channels,
        J = codec_config.J,
        Ne = codec_config.Ne,
        Nd = codec_config.Nd,
        latent_dim = codec_config.latent_dim,
        latent_bits = codec_config.latent_bits,
        lightweight_encode = codec_config.lightweight_encode
    )
    codec.load_state_dict(checkpoint['model_state_dict'])
    codec = codec.to(device)
    codec.eval();
    fic_ds.append(
        ds.map(walloc_compress)
    )

In [6]:
model_id = "google/paligemma-3b-ft-docvqa-896"
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="cuda",
    revision="bfloat16",
)
processor = AutoProcessor.from_pretrained(model_id)

/home/dgj335/.local/lib/python3.10/site-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
def compute_score(item):
    prompt = item['question']
    image = item['image']
    model_inputs = processor(text=prompt, images=image, return_tensors="pt").to(model.device)
    input_len = model_inputs["input_ids"].shape[-1]
    with torch.inference_mode():
        generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
        generation = generation[0][input_len:]
        pred = processor.decode(generation, skip_special_tokens=True)
    score, _ = anls_score(item['answer'], pred, return_gt=True)
    return score, pred

In [8]:
scores = []
preds = []
qid = []
for ds in fic_ds:
    scores.append([])
    preds.append([])
    qid.append([])
    for item in progress_bar(ds):
        score, pred = compute_score(item)
        scores[-1].append(score)
        preds[-1].append(pred)
        qid[-1].append(item['questionId'])

/home/dgj335/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Attempt to open cnn_infer failed: handle=0 error: libcudnn_cnn_infer.so.8: cannot open shared object file: No such file or directory (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:78.)
  return F.conv2d(input, weight, bias, self.stride,
/home/dgj335/.local/lib/python3.10/site-packages/anls_star/anls_star.py:513: UserWarning: Treating ground truth as a list of options. This is a compatibility mode for ST-VQA-like datasets.
  warnings.warn(


In [9]:
fic_ds[0] = fic_ds[0].add_column('preds_16x', preds[0])
fic_ds[1] = fic_ds[1].add_column('preds_4x', preds[1])

In [10]:
fic_ds[0].push_to_hub("danjacobellis/docvqa_walloc_16x",split='test')

Uploading the dataset shards:   0%|          | 0/8 [00:00<?, ?it/s]

Map:   0%|          | 0/649 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Map:   0%|          | 0/649 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/648 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Map:   0%|          | 0/648 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/danjacobellis/docvqa_walloc_16x/commit/ff832ef4c1372e461555f653f1e0e929961be1a6', commit_message='Upload dataset', commit_description='', oid='ff832ef4c1372e461555f653f1e0e929961be1a6', pr_url=None, pr_revision=None, pr_num=None)

In [11]:
fic_ds[1].push_to_hub("danjacobellis/docvqa_walloc_4x",split='test')

Uploading the dataset shards:   0%|          | 0/8 [00:00<?, ?it/s]

Map:   0%|          | 0/649 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Map:   0%|          | 0/649 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/648 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Map:   0%|          | 0/648 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/danjacobellis/docvqa_walloc_4x/commit/72303b4e9e10c8e3c92b1e758e2916dbe4ad94ae', commit_message='Upload dataset', commit_description='', oid='72303b4e9e10c8e3c92b1e758e2916dbe4ad94ae', pr_url=None, pr_revision=None, pr_num=None)